In [1]:
import os
import pandas as pd
import numpy as np
import re

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)

In [2]:
pwdf = pd.read_csv('/home/laura/ANALYSIS/VARIANT_CALLING/COVID/Compare/2020-10-30_Compare_rush/2020-10-30_Compare_rush.snp.pairwise.tsv', sep="\t" , names=['sample_1', 'sample_2', 'dist'])

In [3]:
pwdf.head()

,sample_1,sample_2,dist
0,20069944,20069944,0
1,20069944,20069551,16
2,20069944,20091271,2
3,20069944,20081614,11
4,20069944,20267853,9


In [4]:
pwdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 524176 entries, 0 to 524175
Data columns (total 3 columns):
sample_1    524176 non-null object
sample_2    524176 non-null object
dist        524176 non-null int64
dtypes: int64(1), object(2)
memory usage: 12.0+ MB


In [5]:
samples = list(set(pwdf.sample_1.tolist()))

In [6]:
len(samples)

724

In [7]:
samples[0:5]

['20371405', '20069966', '20326476', '20358289', '20371315']

In [40]:
wgs = pd.read_csv('/home/laura/ANALYSIS/MISC/microreact/WGS_COVID.csv')

In [41]:
wgs.head()

,id,full_date,patient,province__autocolor,SEX__autocolor,AGE__autocolor,day,month,year
0,20060161,09/03/2020,174,Madrid,H,84,9,3,2020
1,20060178,09/03/2020,175,Madrid,H,76,9,3,2020
2,20062236,10/03/2020,176,Madrid,M,96,10,3,2020
3,20062563,10/03/2020,177,Madrid,H,78,10,3,2020
4,20063795,12/03/2020,178,Madrid,H,85,12,3,2020


In [10]:
wgs.shape

(644, 9)

In [11]:
samples_wgs = wgs.id.tolist()

In [12]:
WGS = set(samples_wgs)
PW = set(samples)

In [13]:
len(PW)

724

In [14]:
missing_wgs = list(PW - WGS)
len(missing_wgs)

202

In [15]:
missing_pw = list(WGS - PW)
len(missing_pw)

122

In [16]:
all_missing = missing_pw + missing_wgs
len(all_missing)

324

In [17]:
wgs.shape

(644, 9)

In [42]:
df = wgs[~wgs.id.isin(missing_pw)]

In [19]:
df.shape

(522, 9)

In [43]:
df2 = pd.DataFrame(missing_wgs, columns=['id'])

In [21]:
df2.head()

,id
0,20371405
1,20411920
2,20371315
3,20214753
4,20358933


In [22]:
df.head()

,id,full_date,patient,province__autocolor,SEX__autocolor,AGE__autocolor,day,month,year
2,20062236,10/03/2020,176,Madrid,M,96,10,3,2020
3,20062563,10/03/2020,177,Madrid,H,78,10,3,2020
4,20063795,12/03/2020,178,Madrid,H,85,12,3,2020
5,20066054,13/03/2020,179,Madrid,H,90,13,3,2020
6,20066411,15/03/2020,180,Madrid,H,78,15,3,2020


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 522 entries, 2 to 643
Data columns (total 9 columns):
id                     522 non-null object
full_date              522 non-null object
patient                522 non-null int64
province__autocolor    522 non-null object
SEX__autocolor         522 non-null object
AGE__autocolor         522 non-null object
day                    522 non-null int64
month                  522 non-null int64
year                   522 non-null int64
dtypes: int64(4), object(5)
memory usage: 40.8+ KB


In [44]:
final_df = df.append(df2).reset_index(drop=True)
final_df = final_df[ ['id', 'full_date', 'patient', 'province__autocolor', 'SEX__autocolor',
       'AGE__autocolor', 'day', 'month', 'year'] ]

In [25]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 724 entries, 0 to 723
Data columns (total 9 columns):
id                     724 non-null object
full_date              522 non-null object
patient                522 non-null float64
province__autocolor    522 non-null object
SEX__autocolor         522 non-null object
AGE__autocolor         522 non-null object
day                    522 non-null float64
month                  522 non-null float64
year                   522 non-null float64
dtypes: float64(4), object(5)
memory usage: 51.0+ KB


In [45]:
metadata = pd.read_csv('/home/laura/ANALYSIS/MISC/microreact/wgs_covid_metadata.csv')

In [46]:
metadata.tail()

,id,DEPARTMENT__autocolor
124,20398287,Nefro
125,20395433,Nefro
126,20365524,Nefro
127,20092763,Reinfeccion_run9
128,20411524,Reinfeccion_run9


In [29]:
metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130 entries, 0 to 129
Data columns (total 2 columns):
id                       130 non-null object
DEPARTMENT__autocolor    130 non-null object
dtypes: object(2)
memory usage: 2.2+ KB


In [30]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 724 entries, 0 to 723
Data columns (total 9 columns):
id                     724 non-null object
full_date              522 non-null object
patient                522 non-null float64
province__autocolor    522 non-null object
SEX__autocolor         522 non-null object
AGE__autocolor         522 non-null object
day                    522 non-null float64
month                  522 non-null float64
year                   522 non-null float64
dtypes: float64(4), object(5)
memory usage: 51.0+ KB


In [47]:
final_df = final_df.fillna(0)
final_df.day = final_df.day.astype('Int64')
final_df.month = final_df.month.astype('Int64')
final_df.year = final_df.year.astype('Int64')
final_df.replace(0, np.nan, inplace=True)

In [48]:
final_df = final_df.merge(metadata, how='outer')

In [33]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 738 entries, 0 to 737
Data columns (total 10 columns):
id                       738 non-null object
full_date                522 non-null object
patient                  522 non-null float64
province__autocolor      522 non-null object
SEX__autocolor           522 non-null object
AGE__autocolor           522 non-null object
day                      522 non-null Int64
month                    522 non-null Int64
year                     522 non-null Int64
DEPARTMENT__autocolor    130 non-null object
dtypes: Int64(3), float64(1), object(6)
memory usage: 65.6+ KB


In [49]:
final_df.tail()

,id,full_date,patient,province__autocolor,SEX__autocolor,AGE__autocolor,day,month,year,DEPARTMENT__autocolor
732,20389861,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Digestivo
733,20247923,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Digestivo
734,20395396,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Digestivo
735,20389969,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nefro
736,20092763,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Reinfeccion_run9


In [50]:
def extract_lineage(sample):
    pango_file = os.path.join('/home/laura/ANALYSIS/VARIANT_CALLING/COVID/Annotation/pangolin', sample + ".lineage.csv")
    if os.path.exists(pango_file):
        df = pd.read_csv(pango_file)
        return df.lineage[0]
    else:
        return None

In [51]:
final_df['Lineage__autocolor'] = final_df.apply(lambda x: extract_lineage(x['id']), axis=1)

In [339]:
query_dario = [20242010,20242093,20242702,20247604,20247878,20250481,20250983,20253374,20253475,20253522,20253595,20255027,20255086,20255313,20255524,20255685,20255915,20255928,20258070,20258214,20258238,20258322,20258440,20261662,20263252,20265521,20267856,20269026,20269341,20269791,20272101,20272242,20273792,20274440,20277245,20277326,20279174,20279180,20279189,20279243,20279283,20286667,20286864,20286872,20288207,20289563,20303453]
query_dario = [str(x) for x in query_dario]


In [343]:
query_dario_2 = [20265521,20269791,20272101,20274440,20277245,20277326,20279174,20279180,20279189]
query_dario_2 = [str(x) for x in query_dario_2]

In [340]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 649 entries, 0 to 648
Data columns (total 11 columns):
id                       649 non-null object
full_date                522 non-null object
patient                  522 non-null float64
province__autocolor      522 non-null object
SEX__autocolor           522 non-null object
AGE__autocolor           522 non-null object
day                      522 non-null Int64
month                    522 non-null Int64
year                     522 non-null Int64
DEPARTMENT__autocolor    101 non-null object
Lineage__autocolor       641 non-null object
dtypes: Int64(3), float64(1), object(7)
memory usage: 82.7+ KB


In [346]:
final_df['Dario__autocolor'] = final_df.apply(lambda x: x['id'] in query_dario, axis=1)
final_df['Dario2__autocolor'] = final_df.apply(lambda x: x['id'] in query_dario_2, axis=1)

In [52]:
final_df.to_csv('/home/laura/ANALYSIS/VARIANT_CALLING/COVID/Compare/2020-10-30_Compare_rush/2020-10-30_Compare_rush_wgs_microreact.csv', index=False)

In [39]:
final_df[final_df.id == '20274440']

,id,full_date,patient,province__autocolor,SEX__autocolor,AGE__autocolor,day,month,year,DEPARTMENT__autocolor,Lineage__autocolor
354,20274440,17/08/2020,422.0,Madrid,F,30,17,8,2020,Geriatria,B.1


In [301]:
final_df.shape

(709, 12)